In [1]:
import sys
import os

# add the parent directory ('BNetzA') to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import numpy as np

from models import *
from Data_Preprocessing.data_preprocessing import *

pd.set_option('display.float_format', '{:.4f}'.format)

# Loading Datasets

In [3]:
# load Data
path_to_excel = "../data/EVS4_20140118_dataV9.xlsx"
df = load_data(path_to_excel, sheet_name="DatasetV9")

In [4]:
# df_test_list = [df_test, df_test_xlog, df_test_xlog_ylog, df_test_ylog, df_test_agg, df_test_agg_log, df_test_non_agg, df_test_group_agg]
# baseline, xlog, xlog ylog, ylog, agg, agg ylog, non agg, group agg
# missing: agg xlog, agg xlog ylog
# missing: non agg xlog, non agg xlog ylog, non agg ylog
# missing: group agg xlog, group agg xlog ylog, group agg ylog

In [5]:
outcome_variables = ["cTOTEXn", "cTOTEXn", "cTOTEXn_log", "cTOTEXn_log", "cTOTEXn", "cTOTEXn_log", "cTOTEXn", "cTOTEXn"]
names = ["Baseline", "XLog", "XLog YLog", "YLog", "Aggregates Only", "Aggregates Only YLog", "Disaggregates Only", "Aggregates N1-4 N5-7"]
outcome_transformation = ["None", "None", "log", "log", "None", "log", "None", "None"]
random_states = [39, 40, 41, 42, 43]
columns = ["Model", "Training RMSE", "Training MAE", "Training MAPE", 
           "Testing RMSE", "Testing MAE", "Testing MAPE"]

# Lasso Regression

In [7]:
lasso_results = pd.DataFrame(columns=columns)
lasso_models = []
lasso_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        lasso_result, lasso_model, lasso_vip = lasso_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Lasso {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        lasso_results = pd.concat([lasso_results, lasso_result], axis=0, ignore_index=True)
        lasso_models.append(lasso_model)
        lasso_vips.append(lasso_vip)


Performing Lasso regression...
Evaluating the model...


            Model Training RMSE Training MAE Training MAPE Testing RMSE  \
0  Lasso Baseline    2731309.38   2035203.94          0.12   4539891.03   

  Testing MAE Testing MAPE  
0  2903265.44         0.14  
Performing Lasso regression...
Evaluating the model...


        Model Training RMSE Training MAE Training MAPE Testing RMSE  \
0  Lasso XLog   74197991.25  33866556.29          1.16  30472127.44   

   Testing MAE Testing MAPE  
0  23205370.94         1.11  
Performing Lasso regression...
Evaluating the model...


             Model Training RMSE Training MAE Training MAPE Testing RMSE  \
0  Lasso XLog YLog   18936841.70   7113588.16          0.12   5285677.70   

  Testing MAE Testing MAPE  
0  3203273.32         0.13  
Performing Lasso regression...
Evaluating the model...


        Model Training RMSE Training MAE Training MAPE Testing RMSE  \
0  Lasso YLog   40279573.12  13119174.42          0.25   7039002.36   

  Test

In [8]:
lasso_results.loc[:, lasso_results.columns != 'Model'] = lasso_results.loc[:, lasso_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lasso_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Lasso Aggregates Only YLog,18648730.2240,7132720.5740,0.1240,11474313.8380,5767560.5120,0.1440
6,Lasso XLog YLog,19829971.6300,7274104.4260,0.1220,14612299.3440,6762032.5860,0.1480
1,Lasso Aggregates Only,7346120.9260,4453508.4280,0.1700,16685105.9380,6693552.0260,0.1820
3,Lasso Baseline,2575815.9540,1942412.2420,0.1140,14796210.0440,7271969.3120,0.3480
4,Lasso Disaggregates Only,2592068.1500,1973005.2640,0.1200,14503388.3360,7203839.9520,0.3620
0,Lasso Aggregates N1-4 N5-7,3902520.2240,2876126.1140,0.1500,14739039.1460,7286246.8540,0.3940
7,Lasso YLog,40772998.9600,12765053.7540,0.2420,1426286147.1240,328380000.2280,0.7800
5,Lasso XLog,62190832.1560,29936685.8260,1.1320,68745023.6080,35368388.9840,1.2940


In [9]:
lasso_results.sort_values(by = ["Model", "Testing MAPE"]).tail(50)

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
7,Lasso Aggregates N1-4 N5-7,3793722.6000,2883379.5300,0.1700,4285078.9600,3110579.7300,0.1800
23,Lasso Aggregates N1-4 N5-7,4773444.9700,3365306.7800,0.1500,12057065.4100,5665450.5700,0.2000
31,Lasso Aggregates N1-4 N5-7,4067980.4700,2834644.9600,0.1300,43757160.2700,18625648.3300,0.2000
39,Lasso Aggregates N1-4 N5-7,3422607.2900,2633712.5000,0.1500,5921199.3700,4308814.1000,0.6900
15,Lasso Aggregates N1-4 N5-7,3454845.7900,2663586.8000,0.1500,7674691.7200,4720741.5400,0.7000
28,Lasso Aggregates Only,5008046.4700,3591346.1900,0.1800,49750612.7400,16274148.5800,0.1500
4,Lasso Aggregates Only,8824328.0900,4918414.8000,0.1700,3740575.1700,2783264.3500,0.1800
36,Lasso Aggregates Only,8165661.4700,4686459.7200,0.1600,5060980.3600,3203160.8400,0.1800
12,Lasso Aggregates Only,7071385.2500,4477632.6100,0.1800,11014568.4300,5005643.7600,0.2000
20,Lasso Aggregates Only,7661183.3500,4593688.8200,0.1600,13858792.9900,6201542.6000,0.2000


# Linear Regression with selected Features from Lasso¶

In [11]:
lr_results = pd.DataFrame(columns=columns)
lr_models = []
lr_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        lr_result, lr_model, lr_vip = lasso_feature_selection_linear_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Linear Regresion {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        lr_results = pd.concat([lr_results, lr_result], axis=0, ignore_index=True)
        lr_models.append(lr_model)
        lr_vips.append(lr_vip)


Performing Lasso regression for feature selection...
Performing Linear regression...
Evaluating the model...


                       Model Training RMSE Training MAE Training MAPE  \
0  Linear Regresion Baseline    2156198.63   1625396.86          0.12   

  Testing RMSE Testing MAE Testing MAPE  
0   4939449.57  3168336.74         0.16  
Performing Lasso regression for feature selection...


C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Performing Linear regression...
Evaluating the model...


                   Model Training RMSE Training MAE Training MAPE  \
0  Linear Regresion XLog   69097752.47  40319206.55          1.90   

  Testing RMSE  Testing MAE Testing MAPE  
0  43097998.47  32975362.02         1.65  
Performing Lasso regression for feature selection...


C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Performing Linear regression...
Evaluating the model...


                        Model Training RMSE Training MAE Training MAPE  \
0  Linear Regresion XLog YLog   19583339.44   6984516.07          0.11   

  Testing RMSE Testing MAE Testing MAPE  
0   5835659.44  3378350.17         0.14  
Performing Lasso regression for feature selection...


...

C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Performing Linear regression...
Evaluating the model...


                                 Model Training RMSE Training MAE  \
0  Linear Regresion Disaggregates Only    2221010.35   1658027.90   

  Training MAPE Testing RMSE Testing MAE Testing MAPE  
0          0.12   7816472.60  5972169.50         0.98  
Performing Lasso regression for feature selection...


C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Performing Linear regression...
Evaluating the model...


                                   Model Training RMSE Training MAE  \
0  Linear Regresion Aggregates N1-4 N5-7    3057683.95   2309020.70   

  Training MAPE Testing RMSE Testing MAE Testing MAPE  
0          0.16  17879866.10  8938117.87         1.29  


C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\ducan\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [12]:
lr_results.loc[:, lr_results.columns != 'Model'] = lr_results.loc[:, lr_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lr_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Linear Regresion Aggregates Only YLog,14677973.3300,6067041.1080,0.1140,9699432.5860,5298720.8000,0.1400
6,Linear Regresion XLog YLog,15345685.9140,6299959.0220,0.1140,15030208.1440,6867786.8940,0.1520
1,Linear Regresion Aggregates Only,6789600.6460,4405321.2560,0.1840,16359456.6640,6706496.7620,0.1980
3,Linear Regresion Baseline,2051665.5240,1587536.0200,0.1180,26143451.1540,10879766.3880,0.4840
4,Linear Regresion Disaggregates Only,2123288.3400,1621571.9800,0.1180,29745202.1300,12349881.1780,0.5460
0,Linear Regresion Aggregates N1-4 N5-7,3323075.7760,2508061.0320,0.1600,16217742.8820,8458170.1880,0.6380
5,Linear Regresion XLog,54432064.9480,34150405.9900,1.6960,89508940.9240,50923828.3020,2.4240
7,Linear Regresion YLog,26041250.9800,9175203.8540,0.1880,inf,10142320547350045689240233885590865790529548774...,24478134620635746182772844188168143481390764669...


# Random Forest Regression

In [14]:
rf_results = pd.DataFrame(columns=columns)
rf_models = []
rf_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        rf_result, rf_model, rf_vip = random_forest_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Random Forest {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        rf_results = pd.concat([rf_results, rf_result], axis=0, ignore_index=True)
        rf_models.append(rf_model)
        rf_vips.append(rf_vip)


Performing Random Forest...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Evaluating the model...


                    Model Training RMSE Training MAE Training MAPE  \
0  Random Forest Baseline   30248617.33   6199937.38          0.09   

  Testing RMSE Testing MAE Testing MAPE  
0   7621079.89  4469284.64         0.17  
Performing Random Forest...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Evaluating the model...


                Model Training RMSE Training MAE Training MAPE Testing RMSE  \
0  Random Forest XLog   30305327.84   6277681.97          0.09   7810148.75   

  Testing MAE Testing MAPE  
0  4522345.53         0.17  
Performing Random Forest...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Evaluating the model...


                     Model Training RMSE Training MAE Training MAPE  \
0  Random Forest XLog YLog   36440603.78   7026397.02          0.07   

  Testing RMSE Testing MAE Testing MAPE  
0   4776229.17  3354260.27  

In [15]:
rf_results.loc[:, rf_results.columns != 'Model'] = rf_results.loc[:, rf_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
rf_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
7,Random Forest YLog,36693302.4640,7167030.3120,0.0700,39793393.2180,12615968.8700,0.1820
6,Random Forest XLog YLog,36467920.7900,7086270.6680,0.0700,38545130.3440,12294038.1220,0.1860
2,Random Forest Aggregates Only YLog,37720341.4260,7369140.6220,0.0720,37824967.0540,11750373.7020,0.1920
0,Random Forest Aggregates N1-4 N5-7,29553159.6620,6340270.4580,0.0800,41580339.8440,13574560.1200,0.2220
1,Random Forest Aggregates Only,29774959.3140,6523046.7580,0.0820,38484881.3920,12521322.5200,0.2220
3,Random Forest Baseline,29180181.5740,6122334.1160,0.0800,36600651.9800,12138023.8520,0.2320
5,Random Forest XLog,29336375.2420,6246264.0060,0.0820,37443336.8180,12519869.8480,0.2340
4,Random Forest Disaggregates Only,28743123.8560,6035507.3100,0.0840,37138590.8720,12468855.2040,0.2840


# Decision Tree

In [17]:
dt_results = pd.DataFrame(columns=columns)
dt_models = []
dt_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        dt_result, dt_model, dt_vip = decision_tree_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Decision Tree {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        dt_results = pd.concat([dt_results, dt_result], axis=0, ignore_index=True)
        dt_models.append(dt_model)
        dt_vips.append(dt_vip)


Performing Decision Tree...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Evaluating the model...


                    Model Training RMSE Training MAE Training MAPE  \
0  Decision Tree Baseline   24277022.35   5034147.92          0.05   

  Testing RMSE Testing MAE Testing MAPE  
0  11146704.70  5017293.90         0.17  
Performing Decision Tree...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Evaluating the model...


                Model Training RMSE Training MAE Training MAPE Testing RMSE  \
0  Decision Tree XLog   24277022.35   5034147.92          0.05  11147848.30   

  Testing MAE Testing MAPE  
0  5037512.42         0.17  
Performing Decision Tree...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Evaluating the model...


                     Model Training RMSE Training MAE Training MAPE  \
0  Decision Tree XLog YLog   24308431.12   4798586.81          0.04   

  Testing RMSE Testing MAE Testing MAPE  
0  12727551.27  6654842.7

In [18]:
dt_results.loc[:, dt_results.columns != 'Model'] = dt_results.loc[:, dt_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
dt_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
3,Decision Tree Baseline,14732790.0080,3109712.3300,0.0360,30017159.0520,11046364.9360,0.2160
5,Decision Tree XLog,14653850.4780,3098652.2000,0.0340,32852911.8800,11860650.1600,0.2180
4,Decision Tree Disaggregates Only,5082944.6140,1144066.4880,0.0240,30410442.1560,10751974.8340,0.2220
7,Decision Tree YLog,28298985.5660,6019625.5460,0.0560,53822603.4780,17095497.8600,0.2440
2,Decision Tree Aggregates Only YLog,29351417.3700,6708681.1120,0.0640,66000628.9300,21318852.9340,0.2460
1,Decision Tree Aggregates Only,16350463.3160,4012843.7580,0.0600,34796633.5400,12521636.3640,0.2580
0,Decision Tree Aggregates N1-4 N5-7,4968053.4160,1113052.9740,0.0180,35691519.4860,13040263.2280,0.2640
6,Decision Tree XLog YLog,33598548.2400,7324825.7300,0.0620,87157855.3420,28943622.3480,0.2860


# Cluster-Based Modeling

In [16]:
cluster_results = pd.DataFrame(columns=columns)
cluster_models_c0 = []
cluster_models_c1 = []

outcome_variables_cbm = ["cTOTEXn", "cTOTEXn_log"]
names_cbm = ["Baseline", "YLog"]
outcome_transformation_cbm = ["None", "log"]
indices = [0, 3]

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(outcome_transformation_cbm)):
        cluster_result, model_c0, model_c1 = cluster_based_modeling(df_train_list[indices[i]], df_test_list[indices[i]], outcome_variables_cbm[i], f"Cluster-Based Modeling {names_cbm[i]}", outcome_transformation = outcome_transformation_cbm[i], random_state = random_state)
        cluster_results = pd.concat([cluster_results, cluster_result], axis=0, ignore_index=True)
        cluster_models_c0.append(model_c0)
        cluster_models_c1.append(model_c1)


Cluster
 0    104
-1     70
Name: count, dtype: int64
Performing Cluster-Based Modeling...
Performing Lasso regression...
Evaluating the model...


                             Model Training RMSE Training MAE Training MAPE  \
0  Cluster-Based Modeling Baseline     882075.12    701492.50          0.08   

  Testing RMSE Testing MAE Testing MAPE  
0   2365944.25  1909514.11         0.14  
Evaluating the model...


   Model Training RMSE Training MAE Training MAPE Testing RMSE Testing MAE  \
0  Lasso     882075.12    701492.50          0.08   2365944.25  1909514.11   

  Testing MAPE  
0         0.14  
Performing Lasso regression...
Evaluating the model...


                             Model Training RMSE Training MAE Training MAPE  \
0  Cluster-Based Modeling Baseline    2655365.01   1972121.61          0.05   

  Testing RMSE Testing MAE Testing MAPE  
0   5479753.39  4568669.98         0.21  
Evaluating the model...


   Model Training RMSE Training MAE Training MAPE Testing RMSE Tes

In [17]:
cluster_results.loc[:, cluster_results.columns != 'Model'] = cluster_results.loc[:, cluster_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
cluster_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
1,Cluster-Based Modeling Baseline_Random Forest_...,3937721.2600,2388153.2100,0.0900,11803949.3000,5317071.3400,0.1300
2,Cluster-Based Modeling YLog_Lasso_Random Forest,36981423.7100,6955069.8300,0.1100,17875542.3950,8817983.6750,0.1450
0,Cluster-Based Modeling Baseline_Lasso_Lasso,2032600.5575,1410583.9525,0.0775,10604719.5700,5455368.2300,0.2400
3,Cluster-Based Modeling YLog_Random Forest_Rand...,34018988.1200,6370965.6067,0.0767,57132434.3733,16252076.9867,0.2833


In [ ]:
# chosen frequency: 1, 2, 4, 3